<a href="https://colab.research.google.com/github/jmlDC/MediaBias-Thesis22-23/blob/SBP-Brims/Notebooks/BERT_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installs

In [1]:
!pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 78.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.7 MB/s eta 0:00:00


In [2]:
!pip install --upgrade accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 15.4 MB/s eta 0:00:00


In [3]:
!pip uninstall -y transformers accelerate
!pip install transformers accelerate
!pip install git+https://github.com/huggingface/accelerate
!pip uninstall -y transformers
!pip install transformers==4.28.0

Found existing installation: transformers 4.30.2
Uninstalling transformers-4.30.2:
  Successfully uninstalled transformers-4.30.2
Found existing installation: accelerate 0.21.0
Uninstalling accelerate-0.21.0:
  Successfully uninstalled accelerate-0.21.0
  Using cached transformers-4.30.2-py3-none-any.whl (7.2 MB)
  Using cached accelerate-0.21.0-py3-none-any.whl (244 kB)
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-grldw5lv
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-grldw5lv
  Resolved https://github.com/huggingface/accelerate to commit 299f3ef8ab40f764f3cde8e854f3fe9d3c047f73
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.22.0.dev0-py3-none-any.whl size=245128 sha256=eeeae3c31f833b9e5bd28a867afe6c1346032c6bd315a5510152a4afcdca1437
  Stored in di

## Token hugging face

In [4]:
from huggingface_hub import notebook_login

notebook_login()

## Mounting

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

dir  = "/content/gdrive/MyDrive/THESIS-MS/MediaBias-Thesis22-23/"

Mounted at /content/gdrive


## Dataset

In [ ]:
df

,code_frames,annotations,label
0,10,Immigrants without HOPE need help entering col...,9
1,5,"But in the eyes of the law, he is an illegal i...",4
2,15,"Reaction to Tancredo, Lamm as predicted",14
3,13,"That, said the congressman, is what always hap...",12
4,1,"$50,000 per entry",0
...,...,...,...
46794,11,Smoking is becoming a social taboo,10
46795,5,Nor does it aid lawyers seeking novel ways to...,4
46796,15,'Ashes to Ashes',14
46797,15,SMOKE SCREEN IS SEEN BEHIND THE SMOKELESS,14


In [16]:
from datasets import Dataset
import pandas as pd

df = pd.read_csv(f'{dir}dataset/Extracted-News-May5-7/MFC_prepared.csv', usecols=["code_frames", "annotations"], header=0)
df.code_frames = df.code_frames.astype(int)

df.drop_duplicates(inplace=True)
df

label_dict = {}
for x in range(15):
    label_dict[x+1] = x

label_dict
df['label'] = df.code_frames.replace(label_dict)
df



,code_frames,annotations,label
0,10,Immigrants without HOPE need help entering col...,9
1,5,"But in the eyes of the law, he is an illegal i...",4
2,15,"Reaction to Tancredo, Lamm as predicted",14
3,13,"That, said the congressman, is what always hap...",12
4,1,"$50,000 per entry",0
...,...,...,...
46794,11,Smoking is becoming a social taboo,10
46795,5,Nor does it aid lawyers seeking novel ways to...,4
46796,15,'Ashes to Ashes',14
46797,15,SMOKE SCREEN IS SEEN BEHIND THE SMOKELESS,14


In [17]:
#OLD
df = pd.DataFrame(df)
dataset = Dataset.from_pandas(df).train_test_split(test_size=0.15)

dataset = dataset.rename_column("annotations", "text")

dataset

DatasetDict({
    train: Dataset({
        features: ['code_frames', 'text', 'label', '__index_level_0__'],
        num_rows: 38704
    })
    test: Dataset({
        features: ['code_frames', 'text', 'label', '__index_level_0__'],
        num_rows: 6831
    })
})

In [11]:
from datasets import Dataset, DatasetDict

import datasets
import pandas as pd

dataset = Dataset.from_pandas(df, split=f'train[{k}%:{k+10}%]' for k in range(0, 100, 10))

test_ds = dataset.load_dataset('test', split=[
    f'train[{k}%:{k+10}%]' for k in range(0, 100, 10)
])
trains_ds = dataset.load_dataset('train', split=[
    f'train[:{k}%]+train[{k+10}%:]' for k in range(0, 100, 10)
])


dataset

# df_train = pd.DataFrame({"a": [1, 2, 3]})
# df_test = pd.DataFrame({"ab": [1, 2, 3]})

# datasets_train_test = DatasetDict({
#     "train": Dataset.from_pandas(df_train),
#     "test": Dataset.from_pandas(df_test)
#     })

AttributeError: ignored

## Prep

In [18]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased",
                                          do_lower_case=False)


In [19]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True,
    max_length=128)

In [20]:
mfc_tokenized = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/38704 [00:00<?, ? examples/s]

Map:   0%|          | 0/6831 [00:00<?, ? examples/s]

In [21]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
import numpy as np


# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions), num_classes=15)
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, multi_class="ovr", average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(eval_pred):

    # preds = p.predictions[0] if isinstance(p.predictions,
    #         tuple) else p.predictions
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    result = multi_label_metrics(
        predictions=predictions,
        labels=labels)
    return result

In [ ]:
import evaluate

metric = evaluate.load("accuracy", "f1")

import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [22]:
id2label, label2id = {},{}
for x in range(15):
    id2label[x] = x
        # {0: "NEGATIVE", 1: "POSITIVE"}
    label2id[x] = x
        # {"NEGATIVE": 0, "POSITIVE": 1}

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer



model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased",
                                                        #    problem_type="multi_label_classification",
                                                           num_labels=15,
                                                           id2label=id2label,
                                                           label2id=label2id
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initi

## Training

In [ ]:
training_args = TrainingArguments(
    output_dir="Bert-SBP-try",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=1, #10
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=mfc_tokenized["train"],
    eval_dataset=mfc_tokenized["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Cloning https://huggingface.co/jmLuis/Bert-SBP-try into local empty directory.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: ignored

In [ ]:
trainer.push_to_hub()

# Metric

In [23]:
from transformers import AutoModelForSequenceClassification
import torch
from transformers import AutoTokenizer


In [27]:
from datasets import load_metric
metric = load_metric('f1', 'accuracy')

In [28]:
model = AutoModelForSequenceClassification.from_pretrained("jmLuis/Bert-SBP")


In [ ]:
# clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

In [ ]:
# for model_inputs, gold_standards in evaluation_dataset:
#     predictions = model(model_inputs)
#     metric.add_batch(references=gold_standards, predictions=predictions)
# metric.compute()

NameError: ignored

In [35]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="Bert-SBP-evaluate",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

import numpy as np
from datasets import load_metric
import evaluate

# metric = load_metric("f1", "accuracy")


# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     return metric.compute(predictions=predictions, references=labels)

f1_metric = evaluate.load("f1")
recall_metric = evaluate.load("recall")
accuracy_metric = evaluate.load("accuracy")

def compute_metrics (eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis = -1)

    results = {}
    results.update(f1_metric.compute(predictions=preds, references = labels, average="micro"))
    results.update(recall_metric.compute(predictions=preds, references = labels, average="micro"))
    results.update(accuracy_metric.compute(predictions=preds, references = labels))
    return results


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    tokenizer=tokenizer,
    train_dataset=mfc_tokenized["train"],
    eval_dataset=mfc_tokenized["test"],
    compute_metrics=compute_metrics,
)

trainer.evaluate()

/content/Bert-SBP-evaluate is already a clone of https://huggingface.co/jmLuis/Bert-SBP-evaluate. Make sure you pull the latest changes with `repo.git_pull()`.


{'eval_loss': 0.27911263704299927,
 'eval_f1': 0.9350021958717611,
 'eval_recall': 0.9350021958717611,
 'eval_accuracy': 0.9350021958717611,
 'eval_runtime': 38.7964,
 'eval_samples_per_second': 176.073,
 'eval_steps_per_second': 5.516}

# Prediction

In [ ]:
from transformers import AutoModelForSequenceClassification
import torch
from transformers import AutoTokenizer


In [ ]:
text = "This was a masterpiece. Not completely faithful to the books, but enthralling from beginning to end. Might be my favorite of the three."
text = 'Immigrants without HOPE need help entering college'
text = 'Nan'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("jmLuis/Bert-trial")
inputs = tokenizer(text, return_tensors="pt")

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("jmLuis/Bert-trial")
with torch.no_grad():
    logits = model(**inputs).logits

In [ ]:
predicted_class_id = logits.argmax().item()
out = model.config.id2label[predicted_class_id]
out+1

15

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("jmLuis/Bert-trial")
model = AutoModelForSequenceClassification.from_pretrained("jmLuis/Bert-trial")


def infer_bert(row):
    inputs = tokenizer(str(row['Text']), return_tensors="pt",  truncation=True)
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_class_id = logits.argmax().item()
    out = model.config.id2label[predicted_class_id]
    return int(out)+1  #adding onw because original label is not zero index, however training is.

In [ ]:
def doPrediction(df, filename, last, save=0):
    for i, row in df.iterrows():
        df.at[i,'Annotate'] = infer_bert(row)
        if(i%100==0):
            print(i)
        if (i%2000==0):
            print('saved', save)
            fn = filename+'-save-'+str(save)+'.csv'
            print(filename)
            df.to_csv(fn, index=False)
            save+=1
    df.to_csv(last, index=False)

## applying in curated dataset

In [ ]:
import pandas as pd
# curated_df = pd.read_csv(f'{dir}Official/curated_Dataset.csv', usecols=["Annotate", "Text"], header=0)
curated_df = pd.read_csv(f'{dir}BERT-curated/Bert_curated_dataset-55.csv', usecols=["Annotate", "Text"], header=0)

curated_df

,Annotate,Text
0,2.0,"DOH logs 1,920 new COVID-19 cases, highest dai..."
1,14.0,"The Philippines on Sunday registered 1,920 ne..."
2,11.0,"According to GMA Integrated News Research, th..."
3,9.0,"It is the fourth straight day of more than 1,..."
4,9.0,Data from the Department of Health's COVID-19...
...,...,...
132092,NaN,"""The Philippines and China also discussed new ..."
132093,NaN,"On Thursday, Lazaro started chairing with Chin..."
132094,NaN,"The conduct of the BCM, a confidence-building ..."
132095,NaN,"In an earlier pronouncement, Lazaro asserted t..."


In [ ]:
curated_df.shape

(132097, 2)

In [ ]:
# Original

# save=0
# for i, row in curated_df.iterrows():
#     curated_df.at[i,'Annotate'] = infer_bert(row)
#     if(i%100==0):
#         print(i)
#     if (i%2000==0):
#         print('saved', save)
#         curated_df.to_csv(f'{dir}BERT-curated/Bert_curated_dataset-{save}.csv', index=False)
#         save+=1


In [ ]:
# Token indices sequence length is longer than the specified maximum sequence length for this model (642 > 512). Running this sequence through the model will result in indexing errors
# RuntimeError: The size of tensor a (642) must match the size of tensor b (512) at non-singleton dimension 1

In [ ]:
# curated_df.to_csv(f'{dir}BERT-curated/Bert_curated_dataset-last.csv', index=False)

In [ ]:
# curated_df.iloc[:29953].to_csv(f'{dir}BERT-curated/Bert_curated_dataset-cut.csv', index=False)

In [ ]:
save=60
for i, row in curated_df.iloc[120000:].iterrows():
    curated_df.at[i,'Annotate'] = infer_bert(row)
    if(i%100==0):
        print(i)
    if (i%2000==0):
        print('saved', save)
        curated_df.to_csv(f'{dir}BERT-curated/Bert_curated_dataset-{save}.csv', index=False)
        save+=1


120000
saved 60
120100
120200
120300
120400
120500
120600
120700
120800
120900
121000
121100
121200
121300
121400
121500
121600
121700
121800
121900
122000
saved 61
122100
122200
122300
122400
122500
122600
122700
122800
122900
123000
123100
123200
123300
123400
123500
123600
123700
123800
123900
124000
saved 62
124100
124200
124300
124400
124500
124600
124700
124800
124900
125000
125100
125200
125300
125400
125500
125600
125700
125800
125900
126000
saved 63
126100
126200
126300
126400
126500
126600
126700
126800
126900
127000
127100
127200
127300
127400
127500
127600
127700
127800
127900
128000
saved 64
128100
128200
128300
128400
128500
128600
128700
128800
128900
129000
129100
129200
129300
129400
129500
129600
129700
129800
129900
130000
saved 65
130100
130200
130300
130400
130500
130600
130700
130800
130900
131000
131100
131200
131300
131400
131500
131600
131700
131800
131900
132000
saved 66


In [ ]:
curated_df.to_csv(f'{dir}BERT-curated/Bert_curated_dataset-last.csv', index=False)

In [ ]:
curated_df.tail()

,Annotate,Text
132092,14.0,"""The Philippines and China also discussed new ..."
132093,14.0,"On Thursday, Lazaro started chairing with Chin..."
132094,8.0,"The conduct of the BCM, a confidence-building ..."
132095,14.0,"In an earlier pronouncement, Lazaro asserted t..."
132096,8.0,This came as China is accused of conducting gr...


## Predict using hedlines and lead MFC

In [ ]:
import pandas as pd
curated_df = pd.read_csv(f'{dir}Official/curated_Dataset_headlines_lead.csv', header=0)

curated_df

,Text,Annotate
0,"DOH logs 1,920 new COVID-19 cases, highest dai...",NaN
1,"The Philippines on Sunday registered 1,920 new...",NaN
2,"DOH logs 9 new COVID-19 deaths; 1,878 fresh ca...",NaN
3,The Department of Health said Saturday that th...,NaN
4,Bettor wins P55.3 million jackpot in Grand Lot...,NaN
...,...,...
21599,The lower chamber recently gave its final appr...,NaN
21600,Immediate expansion of gov’t subsidies to kind...,NaN
21601,A network of advocates called for the immediat...,NaN
21602,"P​H, China vow UNCLOS​ compliance, seek Indo-P...",NaN


In [ ]:
save=0
for i, row in curated_df.iterrows():
    curated_df.at[i,'Annotate'] = infer_bert(row)
    if(i%100==0):
        print(i)
    if (i%2000==0):
        print('saved', save)
        curated_df.to_csv(f'{dir}predictions-HL/BERT/Bert_curated_dataset-{save}.csv', index=False)
        save+=1


0
saved 0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
saved 1
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
saved 2
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
saved 3
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
saved 4
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
saved 5
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
saved 6
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
saved 7
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
1750

In [ ]:
curated_df.to_csv(f'{dir}predictions-HL/BERT/Bert_curated_dataset-last.csv', index=False)

## Predicting per Outlet

### GMA

In [ ]:
import pandas as pd
gma = pd.read_csv(f'{dir}dataset/cleaned_newsArticles/gma-HL.csv', header=0)
gma

,Text,Annotate
0,"DOH logs 1,920 new COVID-19 cases, highest dai...",NaN
1,"The Philippines on Sunday registered 1,920 new...",NaN
2,"DOH logs 9 new COVID-19 deaths; 1,878 fresh ca...",NaN
3,The Department of Health said Saturday that th...,NaN
4,Bettor wins P55.3 million jackpot in Grand Lot...,NaN
...,...,...
4406,The camp of detained former Senator Leila de L...,NaN
4407,Philippines to appeal ICC’s resumption of drug...,NaN
4408,The Philippine government intends to appeal th...,NaN
4409,Reported death toll from bad weather now at 40...,NaN


In [ ]:
doPrediction(gma, filename=f'{dir}predictions/SBP/gmaBERT' ,last=f'{dir}predictions/SBP/gmaBERT-last.csv', save =2, start=4000)

4000
saved 2
/content/gdrive/MyDrive/THESIS-MS/MediaBias-Thesis22-23/predictions/SBP/gmaBERT-save-2.csv
4100
4200
4300
4400


### PNA

In [ ]:
pna = pd.read_csv(f'{dir}dataset/cleaned_newsArticles/pna-HL.csv', header=0)
pna

,Text,Annotate
0,"MIAA, Meralco begins NAIA T3 electrical audit",NaN
1,MANILA – The Manila International Airport Auth...,NaN
2,"Pag-IBIG, Singaporean firm eye building ‘net-z...",NaN
3,"MANILA – Marilene Acosta, the CEO of Pag-IBIG ...",NaN
4,DMW assures aid for OFWS from Sudan who wanted...,NaN
...,...,...
4395,"MANILA – More than 1,200 police officers have ...",NaN
4396,"IP leaders call for unity to preserve IPRA, NC...",NaN
4397,MANILA – Leaders of the indigenous peoples (IP...,NaN
4398,"PH, US Navy fleets to tackle cooperation, exch...",NaN


In [ ]:
doPrediction(pna, filename=f'{dir}predictions/SBP/pnaBERT' ,last=f'{dir}predictions/SBP/pnaBERT-last.csv')

0
saved 0
/content/gdrive/MyDrive/THESIS-MS/MediaBias-Thesis22-23/predictions/SBP/pnaBERT
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
saved 1
/content/gdrive/MyDrive/THESIS-MS/MediaBias-Thesis22-23/predictions/SBP/pnaBERT
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
saved 2
/content/gdrive/MyDrive/THESIS-MS/MediaBias-Thesis22-23/predictions/SBP/pnaBERT
4100
4200
4300


### Rappler

In [ ]:
rappler = pd.read_csv(f'{dir}dataset/cleaned_newsArticles/rappler-HL.csv', header=0)
rappler

,Text,Annotate
0,Marcos the party-going schoolboy returns to UK...,NaN
1,"LONDON, United Kingdom – In the late ’70s, Fer...",NaN
2,"Ablan confirms resignation from DepEd, denies ...",NaN
3,"MANILA, Philippines – Former Department of Edu...",NaN
4,WATCH: Paano naibenta sa labas ang DepEd lapto...,NaN
...,...,...
4394,"MANILA, Philippines – The Philippines is looki...",NaN
4395,"3 Cordillera provinces, Northern Samar placed ...",NaN
4396,"MANILA, Philippines – The alert level of three...",NaN
4397,Comelec starts printing ballots for 2022 polls,NaN


In [ ]:
doPrediction(rappler, filename=f'{dir}predictions/SBP/rapplerBERT' ,last=f'{dir}predictions/SBP/rapplerBERT-last.csv')

0
saved 0
/content/gdrive/MyDrive/THESIS-MS/MediaBias-Thesis22-23/predictions/SBP/rapplerBERT
100
200
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
saved 1
/content/gdrive/MyDrive/THESIS-MS/MediaBias-Thesis22-23/predictions/SBP/rapplerBERT
2100
2200
2300
2400
2500
2600
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
saved 2
/content/gdrive/MyDrive/THESIS-MS/MediaBias-Thesis22-23/predictions/SBP/rapplerBERT
4100
4200
4300


### Sunstar

In [ ]:
sunstar = pd.read_csv(f'{dir}dataset/cleaned_newsArticles/sunstar-HL.csv', header=0)
sunstar

,Text,Annotate
0,Sandiganbayan affirms denial of motion seeking...,NaN
1,THE Sandiganbayan has affirmed the decision of...,NaN
2,PH to reassess protocols with end of Covid glo...,NaN
3,WITH the end of the global emergency status fo...,NaN
4,DMW: Work experience top criterion for oversea...,NaN
...,...,...
4516,CITY OF SAN FERNANDO More than two kilos of su...,NaN
4517,Crissot's 156 birth anniversary commemorated,NaN
4518,BACOLOR The heirs of Juan Crisostomo Soto (Cri...,NaN
4519,CSF council inquiry on power rate hike launched,NaN


In [ ]:
doPrediction(sunstar, filename=f'{dir}predictions/SBP/sunstarBERT' ,last=f'{dir}predictions/SBP/sunstarBERT-last.csv')

0
saved 0
/content/gdrive/MyDrive/THESIS-MS/MediaBias-Thesis22-23/predictions/SBP/sunstarBERT
100
200
300
400
500
600
700
800
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
saved 1
/content/gdrive/MyDrive/THESIS-MS/MediaBias-Thesis22-23/predictions/SBP/sunstarBERT
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3500
3600
3700
3800
3900
4000
saved 2
/content/gdrive/MyDrive/THESIS-MS/MediaBias-Thesis22-23/predictions/SBP/sunstarBERT
4100
4200
4300
4400
4500


### MB

In [ ]:
mb = pd.read_csv(f'{dir}dataset/cleaned_newsArticles/mb-HL.csv', header=0)
mb

,Text,Annotate
0,'Thank you gov': Cong Roman mourns passing of ...,NaN
1,Bataan 1st district Rep. Geraldine Roman paid ...,NaN
2,P5-M reward offered to find missing Chinese pa...,NaN
3,A P5-million reward was offered to anyone who ...,NaN
4,"Hontiveros: Nuclear energy a terrible, false s...",NaN
...,...,...
3868,The lower chamber recently gave its final appr...,NaN
3869,Immediate expansion of gov’t subsidies to kind...,NaN
3870,A network of advocates called for the immediat...,NaN
3871,"P​H, China vow UNCLOS​ compliance, seek Indo-P...",NaN


In [ ]:
doPrediction(mb, filename=f'{dir}predictions/SBP/mbBERT' ,last=f'{dir}predictions/SBP/mbBERT-last.csv')

0
saved 0
/content/gdrive/MyDrive/THESIS-MS/MediaBias-Thesis22-23/predictions/SBP/mbBERT
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
saved 1
/content/gdrive/MyDrive/THESIS-MS/MediaBias-Thesis22-23/predictions/SBP/mbBERT
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800


## Predicting per Outlet similar issues

### Gma

In [ ]:
import pandas as pd
gma = pd.read_csv(f'{dir}dataset/cleaned_newsArticles/gma-spec.csv', header=0)
gma

,Text,Annotate
0,Fishermen protest no-sail zone policy amid Bal...,NaN
1,Fishermen in Zambales protested on Wednesday ...,NaN
2,"“Ang buhay kasi ng mangingisda eh isang, kahi...",NaN
3,(The life of a fisherman is hard. The impact ...,NaN
4,De Jesus was also concerned about the effects...,NaN
...,...,...
125,Committee chairperson Senator Imee Marcos sai...,NaN
126,"Aguilar, however, maintained that the selecti...",NaN
127,“Ito ay ayon siyempre sa mga nangyayaring dev...,NaN
128,(This was according to the developments happe...,NaN


In [ ]:
doPrediction(gma, filename=f'{dir}predictions/SBP/gmaBERT-specIssue' ,last=f'{dir}predictions/SBP/gmaBERT-specIssue-last.csv')

0
saved 0
/content/gdrive/MyDrive/THESIS-MS/MediaBias-Thesis22-23/predictions/SBP/gmaBERT-specIssue
100


### PNA

In [ ]:
pna = pd.read_csv(f'{dir}dataset/cleaned_newsArticles/pna-spec.csv', header=0)
pna

,Text,Annotate
0,First batch of 17 Filipino evacuees from Sudan...,NaN
1,"MANILA – Government officials, led by Departme...",NaN
2,The group arrived past 6:30 p.m. at Terminal 3...,NaN
3,Among the evacuees from conflict-torn Sudan ar...,NaN
4,A report from the Department of Social Welfare...,NaN
...,...,...
408,The SAF said in a statement on Facebook that a...,NaN
409,"""We are waiting for the process to begin in h...",NaN
410,It added that the Saudi diplomatic mission was...,NaN
411,Kuwait also said Saturday that it had launched...,NaN


In [ ]:
doPrediction(pna, filename=f'{dir}predictions/SBP/pnaBERT-specIssue' ,last=f'{dir}predictions/SBP/pnaBERT-specIssue-last.csv')

0
saved 0
/content/gdrive/MyDrive/THESIS-MS/MediaBias-Thesis22-23/predictions/SBP/pnaBERT-specIssue
100
200
300
400


### Rappler

In [ ]:
rappler = pd.read_csv(f'{dir}dataset/cleaned_newsArticles/rappler-spec.csv', header=0)
rappler

,Text,Annotate
0,First batch of Filipino evacuees from Sudan re...,NaN
1,"MANILA, Philippines – The first batch of Filip...",NaN
2,The 17 Filipino evacuees from the African coun...,NaN
3,"According to the DFA, eight of the rescued Fil...",NaN
4,"As I the 1st batch of repatriates from Sudan,...",NaN
...,...,...
123,“Iba-iba po kasi ang situation ng ating mga pa...,NaN
124,(Each school has a different situation. That’s...,NaN
125,Schools in the country have reverted to in-per...,NaN
126,Weather bureau PAGASA had warned that the coun...,NaN


In [ ]:
doPrediction(rappler, filename=f'{dir}predictions/SBP/rapplerBERT-specIssue' ,last=f'{dir}predictions/SBP/rapplerBERT-specIssue-last.csv')

0
saved 0
/content/gdrive/MyDrive/THESIS-MS/MediaBias-Thesis22-23/predictions/SBP/rapplerBERT-specIssue
100


### Sunstar

In [ ]:
sunstar = pd.read_csv(f'{dir}dataset/cleaned_newsArticles/sunstar-spec.csv', header=0)
sunstar

,Text,Annotate
0,DMW to charter flight for repatriation of 340 ...,NaN
1,THREE hundred forty overseas Filipino workers ...,NaN
2,"In an online press briefing, DMW Secretary Sus...",NaN
3,"""We are now exploring the possibility of a cha...",NaN
4,So certain arrangements will have to be made b...,NaN
...,...,...
191,"""Balikatan"" being held from April 11 to 28 thi...",NaN
192,Castillo said that pilots from PAF's 5th Fight...,NaN
193,TAI is a military operation that involves laun...,NaN
194,"""The primary objective of this operation is to...",NaN


In [ ]:
doPrediction(sunstar, filename=f'{dir}predictions/SBP/sunstarBERT-specIssue' ,last=f'{dir}predictions/SBP/sunstarBERT-specIssue-last.csv')

0
saved 0
/content/gdrive/MyDrive/THESIS-MS/MediaBias-Thesis22-23/predictions/SBP/sunstarBERT-specIssue
100


### Manila Bulletin

In [ ]:
mb = pd.read_csv(f'{dir}dataset/cleaned_newsArticles/mb-spec.csv', header=0)
mb

,Text,Annotate
0,F​ilipino evacuees from Sudan start arriving i...,NaN
1,The first batch of Filipinos who fled from Sud...,NaN
2,Department of Foreign Affairs (DFA) Secretary ...,NaN
3,"Nine of them, including an infant, got out of ...",NaN
4,The Philippine consulate received them on Apri...,NaN
...,...,...
419,The President said the government was working ...,NaN
420,"""We are just waiting to get better information...",NaN
421,The in-fighting between the Sudanese Armed For...,NaN
422,According to the World Health Organization (WH...,NaN


In [ ]:
doPrediction(mb, filename=f'{dir}predictions/SBP/mbBERT-specIssue' ,last=f'{dir}predictions/SBP/mbBERT-specIssue-last.csv')

0
saved 0
/content/gdrive/MyDrive/THESIS-MS/MediaBias-Thesis22-23/predictions/SBP/mbBERT-specIssue
100
200
300
400


# Others

In [ ]:
#REference fine tuning
# https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/text_classification.ipynb